<a href="https://colab.research.google.com/github/doristw/bads/blob/master/Channel_Price_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests beautifulsoup4

In [3]:
from google.colab import files
import pandas as pd

# Step 1: Upload the file
uploaded = files.upload()

# Step 2: Load the file into a DataFrame
file_name = 'Channel url data.xlsx'  # Replace with your file name
df_url = pd.read_excel(file_name)

# Display the DataFrame
print(df_url)



Saving Channel url data.xlsx to Channel url data.xlsx
      Channel                 Product  \
0    Coolblue        OnePlus 12 256GB   
1    Coolblue        OnePlus 12 512GB   
2    Coolblue        OnePlus 12 512GB   
3    Coolblue             OnePlus 12R   
4    Coolblue             OnePlus 12R   
5    Coolblue   OnePlus Nord 3 128 GB   
6    Coolblue   OnePlus Nord 3 128 GB   
7    Coolblue   OnePlus Nord 3 256 GB   
8    Coolblue   OnePlus Nord 3 256 GB   
9    Coolblue   OnePlus Nord CE3 Lite   
10   Coolblue   OnePlus Nord CE3 Lite   
11   Coolblue       OnePlus Nord CE 4   
12  Mobiel.nl        OnePlus 12 256GB   
13  Mobiel.nl        OnePlus 12 512GB   
14  Mobiel.nl             OnePlus 12R   
15  Mobiel.nl             OnePlus 12R   
16  Mobiel.nl   OnePlus Nord 3 128 GB   
17  Mobiel.nl    OnePlus Nord 3 128GB   
18  Mobiel.nl  OnePlus Nord CE 3 Lite   
19  Mobiel.nl   OnePlus Nord CE4 Lite   
20  Mobiel.nl   OnePlus Nord CE4 Lite   
21    bol.com        OnePlus 12 512GB   
22 

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_coolblue_product_price(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        product_price = soup.find('strong', class_='sales-price__current').text.strip()
        return product_price.replace(',-', '')
    except Exception as e:
        print(f"Failed to retrieve the price from Coolblue for URL {url}: {e}")
        return None

def get_mobiel_product_price(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        price_container = soup.find('div', class_='PriceTag__Container-sc-cf4af607-0')
        if price_container:
            product_price = price_container.find('span').text.strip()
            return product_price.replace('-', '')
        else:
            print(f"Price container not found for Mobiel.nl URL {url}")
            return None
    except Exception as e:
        print(f"Failed to retrieve the price from Mobiel.nl for URL {url}: {e}")
        return None

def get_bol_product_price(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        price_container = soup.find('span', class_='promo-price')
        if price_container:
            product_price = price_container.text.strip().replace('\n', '').replace(' ', '').replace('-', '')
            return product_price
        else:
            print(f"Price container not found for Bol.com URL {url}")
            return None
    except Exception as e:
        print(f"Failed to retrieve the price from Bol.com for URL {url}: {e}")
        return None

def get_product_price(row):
    url = row['url']
    channel = row['Channel']
    if channel == 'Coolblue':
        return get_coolblue_product_price(url)
    elif channel == 'Mobiel.nl':
        return get_mobiel_product_price(url)
    elif channel == 'bol.com':
        return get_bol_product_price(url)
    else:
        return None

# Assuming df_url is your DataFrame containing the URLs
df_url = pd.read_excel('Channel url data.xlsx')

# Apply the function to get prices
df_url['Price'] = df_url.apply(get_product_price, axis=1)

# Display the DataFrame with the prices
print(df_url[['Channel', 'SKU', 'Price']])

# Optionally, save the DataFrame to a CSV file
df_url.to_csv('product_prices_with_channels.csv', index=False)


      Channel                                                SKU Price
0    Coolblue                          OnePlus 12 256GB Black 5G   849
1    Coolblue                          OnePlus 12 512GB Green 5G   899
2    Coolblue                          OnePlus 12 512GB Black 5G   899
3    Coolblue                          OnePlus 12R 256GB Gray 5G   549
4    Coolblue                          OnePlus 12R 256GB Blue 5G   549
5    Coolblue                       OnePlus Nord 3 128GB Gray 5G   329
6    Coolblue                      OnePlus Nord 3 128GB Green 5G   329
7    Coolblue                       OnePlus Nord 3 256GB Gray 5G   399
8    Coolblue                      OnePlus Nord 3 256GB Green 5G   399
9    Coolblue                OnePlus Nord CE3 Lite 128GB Gray 5G   179
10   Coolblue               OnePlus Nord CE3 Lite 128GB Green 5G   179
11   Coolblue              OnePlus Nord CE 4 Lite 256GB Blauw 5G   279
12  Mobiel.nl                                   OnePlus 12 256GB   827
13  Mo